In [1]:
import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import random as rn
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
import codecs

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# load feature and labels
feat = np.load('../../dimensionalSM/data/feat_34_hfs.npy')
vad = np.load('../../dimensionalSM/data/y_egemaps.npy')
# remove outlier, < 1, > 5
vad = np.where(vad==5.5, 5.0, vad)
vad = np.where(vad==0.5, 1.0, vad)

# standardization
scaled_feature = False

In [4]:
#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
data = np.load('../../dimensionalSM/IEMOCAP_full_release/data_collected_10039.pickle', allow_pickle=True)
text = [t['transcription'] for t in data]
#vad = np.load('../../data/y_egemaps.npy')
# load fastext model
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_train_text = []
x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
nb_words = len(word_index) +1
print('Found %s unique tokens' % len(word_index))

# load model
EMBEDDING_DIM = 300
file_loc = '../../dimensionalSM/data/crawl-300d-2M-subword.vec'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))


earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)


Found 3437 unique tokens
../../dimensionalSM/data/crawl-300d-2M-subword.vec
G Word embeddings: 2000000
G Null word embeddings: 133


In [5]:
if scaled_feature == True:
    scaler = StandardScaler()
    scaler = scaler.fit(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaler.transform(feat.reshape(feat.shape[0]*feat.shape[1], feat.shape[2]))
    scaled_feat = scaled_feat.reshape(feat.shape[0], feat.shape[1], feat.shape[2])
    feat = scaled_feat
else:
    feat = feat

scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad

# Concordance correlation coefficient (CCC)-based loss function - using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1, keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss


In [6]:
# split train/test
split = 7869

In [7]:
# API model
def model():
    # speech network
    input_speech = Input(shape=(feat.shape[1], feat.shape[2]), name='speech_input')
    net_speech = BatchNormalization()(input_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(128, return_sequences=True)(net_speech)
    net_speech = LSTM(64, return_sequences=True)(net_speech)
    net_speech = LSTM(32, return_sequences=True)(net_speech)
    model_speech = Flatten()(net_speech)
    #model_speech = Dropout(0.1)(net_speech)
    
    #text network
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net_text = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(input_text)
    net = Bidirectional(LSTM(128, return_sequences=True))(net_text)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    model_text = Dense(64)(net)

    # combined model
    model_combined = concatenate([model_speech, model_text])
    model_combined = Dense(64, activation='relu')(model_combined)
    model_combined = Dense(32, activation='relu')(model_combined)
    #model_combined = Dropout(0.4)(model_combined)
    target_names = ('v', 'a', 'd')
    model_combined = [Dense(1, name=name)(model_combined) for name in target_names]

    model = Model([input_speech, input_text], model_combined) 
    model.compile(loss=ccc_loss,
                  loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model_1 = model()
# model_1.summary()

In [8]:
# main function for for LOSO
def main():
    model_1 = model()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [feat[:split],x_train_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [feat[split:],x_train_text[split:]], vad[split:].T.tolist())
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(30):
        best = main()
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('speech_textFASTTEXT_multi7869.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 46s 7ms/step - loss: 1.4751 - v_loss: 0.5744 - a_loss: 0.4076 - d_loss: 0.4924 - v_ccc: 0.4251 - a_ccc: 0.5922 - d_ccc: 0.5076 - val_loss: 1.9772 - val_v_loss: 0.6418 - val_a_loss: 0.7098 - val_d_loss: 0.6386 - val_v_ccc: 0.3668 - val_a_ccc: 0.2935 - val_d_ccc: 0.3624
Epoch 2/50
6295/6295 [==============================] - 48s 8ms/step - loss: 1.0689 - v_loss: 0.3517 - a_loss: 0.3166 - d_loss: 0.4008 - v_ccc: 0.6484 - a_ccc: 0.6834 - d_ccc: 0.5993 - val_loss: 1.6279 - val_v_loss: 0.6095 - val_a_loss: 0.5031 - val_d_loss: 0.5230 - val_v_ccc: 0.3958 - val_a_ccc: 0.5003 - val_d_ccc: 0.4759
Epoch 3/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.9597 - v_loss: 0.2970 - a_loss: 0.2936 - d_loss: 0.3687 - v_ccc: 0.7029 - a_ccc: 0.7061 - d_ccc: 0.6313 - val_loss: 1.5444 - val_v_loss: 0.60

Epoch 9/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.6357 - v_loss: 0.1758 - a_loss: 0.2130 - d_loss: 0.2469 - v_ccc: 0.8241 - a_ccc: 0.7868 - d_ccc: 0.7534 - val_loss: 1.5681 - val_v_loss: 0.5503 - val_a_loss: 0.4655 - val_d_loss: 0.5701 - val_v_ccc: 0.4577 - val_a_ccc: 0.5387 - val_d_ccc: 0.4356
Epoch 10/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.5969 - v_loss: 0.1619 - a_loss: 0.1990 - d_loss: 0.2359 - v_ccc: 0.8381 - a_ccc: 0.8011 - d_ccc: 0.7640 - val_loss: 1.5821 - val_v_loss: 0.5511 - val_a_loss: 0.4702 - val_d_loss: 0.5773 - val_v_ccc: 0.4585 - val_a_ccc: 0.5296 - val_d_ccc: 0.4298
Epoch 11/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.5537 - v_loss: 0.1561 - a_loss: 0.1833 - d_loss: 0.2140 - v_ccc: 0.8438 - a_ccc: 0.8166 - d_ccc: 0.7859 - val_loss: 1.5559 - val_v_loss: 0.5398 - val_a_loss: 0.4645 - val_d_loss: 0.5703 - val_v_ccc: 0.4696 - val_a_ccc: 0.5398 - val_d_ccc: 0.4347
Epoch 12/50
6295/6295 [

6295/6295 [==============================] - 50s 8ms/step - loss: 1.0861 - v_loss: 0.3543 - a_loss: 0.3230 - d_loss: 0.4090 - v_ccc: 0.6456 - a_ccc: 0.6773 - d_ccc: 0.5909 - val_loss: 1.6393 - val_v_loss: 0.6385 - val_a_loss: 0.5038 - val_d_loss: 0.5083 - val_v_ccc: 0.3684 - val_a_ccc: 0.4997 - val_d_ccc: 0.4925
Epoch 3/50
6295/6295 [==============================] - 50s 8ms/step - loss: 0.9755 - v_loss: 0.3031 - a_loss: 0.3007 - d_loss: 0.3725 - v_ccc: 0.6968 - a_ccc: 0.6996 - d_ccc: 0.6281 - val_loss: 1.5732 - val_v_loss: 0.6027 - val_a_loss: 0.4413 - val_d_loss: 0.5459 - val_v_ccc: 0.4053 - val_a_ccc: 0.5622 - val_d_ccc: 0.4593
Epoch 4/50
6295/6295 [==============================] - 50s 8ms/step - loss: 0.8957 - v_loss: 0.2640 - a_loss: 0.2828 - d_loss: 0.3487 - v_ccc: 0.7359 - a_ccc: 0.7171 - d_ccc: 0.6513 - val_loss: 1.5282 - val_v_loss: 0.6069 - val_a_loss: 0.4264 - val_d_loss: 0.5103 - val_v_ccc: 0.4014 - val_a_ccc: 0.5784 - val_d_ccc: 0.4920
Epoch 5/50
6295/6295 [==============

Epoch 10/50
6295/6295 [==============================] - 52s 8ms/step - loss: 0.6041 - v_loss: 0.1657 - a_loss: 0.2043 - d_loss: 0.2347 - v_ccc: 0.8346 - a_ccc: 0.7958 - d_ccc: 0.7654 - val_loss: 1.6038 - val_v_loss: 0.5659 - val_a_loss: 0.4728 - val_d_loss: 0.5852 - val_v_ccc: 0.4432 - val_a_ccc: 0.5325 - val_d_ccc: 0.4205
Epoch 11/50
6295/6295 [==============================] - 51s 8ms/step - loss: 0.5692 - v_loss: 0.1593 - a_loss: 0.1936 - d_loss: 0.2164 - v_ccc: 0.8407 - a_ccc: 0.8065 - d_ccc: 0.7836 - val_loss: 1.6146 - val_v_loss: 0.5634 - val_a_loss: 0.4654 - val_d_loss: 0.6029 - val_v_ccc: 0.4447 - val_a_ccc: 0.5364 - val_d_ccc: 0.4043
Epoch 12/50
6295/6295 [==============================] - 51s 8ms/step - loss: 0.5366 - v_loss: 0.1473 - a_loss: 0.1822 - d_loss: 0.2070 - v_ccc: 0.8528 - a_ccc: 0.8177 - d_ccc: 0.7930 - val_loss: 1.6100 - val_v_loss: 0.5563 - val_a_loss: 0.4811 - val_d_loss: 0.5831 - val_v_ccc: 0.4501 - val_a_ccc: 0.5166 - val_d_ccc: 0.4233
Epoch 13/50
6295/6295 

6295/6295 [==============================] - 49s 8ms/step - loss: 0.7607 - v_loss: 0.2167 - a_loss: 0.2459 - d_loss: 0.2979 - v_ccc: 0.7833 - a_ccc: 0.7540 - d_ccc: 0.7020 - val_loss: 1.5495 - val_v_loss: 0.5710 - val_a_loss: 0.4241 - val_d_loss: 0.5634 - val_v_ccc: 0.4305 - val_a_ccc: 0.5778 - val_d_ccc: 0.4421
Epoch 7/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.7159 - v_loss: 0.1983 - a_loss: 0.2344 - d_loss: 0.2837 - v_ccc: 0.8021 - a_ccc: 0.7656 - d_ccc: 0.7164 - val_loss: 1.6116 - val_v_loss: 0.5886 - val_a_loss: 0.4657 - val_d_loss: 0.5730 - val_v_ccc: 0.4213 - val_a_ccc: 0.5348 - val_d_ccc: 0.4323
Epoch 8/50
6295/6295 [==============================] - 49s 8ms/step - loss: 0.6748 - v_loss: 0.1882 - a_loss: 0.2212 - d_loss: 0.2654 - v_ccc: 0.8117 - a_ccc: 0.7789 - d_ccc: 0.7346 - val_loss: 1.5522 - val_v_loss: 0.5647 - val_a_loss: 0.4478 - val_d_loss: 0.5593 - val_v_ccc: 0.4474 - val_a_ccc: 0.5548 - val_d_ccc: 0.4457
Epoch 9/50
6295/6295 [==============

...

Epoch 14/50
2170/2170 [==============================] - 5s 2ms/step
23
Train on 6295 samples, validate on 1574 samples
Epoch 1/50
6295/6295 [==============================] - 67s 11ms/step - loss: 1.5118 - v_loss: 0.5891 - a_loss: 0.4099 - d_loss: 0.5132 - v_ccc: 0.4107 - a_ccc: 0.5902 - d_ccc: 0.4873 - val_loss: 2.0652 - val_v_loss: 0.6531 - val_a_loss: 0.7178 - val_d_loss: 0.6947 - val_v_ccc: 0.3470 - val_a_ccc: 0.2839 - val_d_ccc: 0.3038
Epoch 2/50
6295/6295 [==============================] - 50s 8ms/step - loss: 1.0798 - v_loss: 0.3549 - a_loss: 0.3189 - d_loss: 0.4064 - v_ccc: 0.6453 - a_ccc: 0.6810 - d_ccc: 0.5938 - val_loss: 1.6352 - val_v_loss: 0.6097 - val_a_loss: 0.5025 - val_d_loss: 0.5333 - val_v_ccc: 0.3935 - val_a_ccc: 0.5026 - val_d_ccc: 0.4687
Epoch 3/50
6295/6295 [==============================] - 50s 8ms/step - loss: 0.9628 - v_loss: 0.2951 - a_loss: 0.2974 - d_loss: 0.3704 - v_ccc: 0.7049 - a_ccc: 0.7026 - d_ccc: 0.6296 - val_loss: 1.5057 - val_v_loss: 0.5803 - val_

6295/6295 [==============================] - 51s 8ms/step - loss: 0.5581 - v_loss: 0.1510 - a_loss: 0.1908 - d_loss: 0.2161 - v_ccc: 0.8490 - a_ccc: 0.8091 - d_ccc: 0.7838 - val_loss: 1.5917 - val_v_loss: 0.5733 - val_a_loss: 0.4710 - val_d_loss: 0.5676 - val_v_ccc: 0.4362 - val_a_ccc: 0.5298 - val_d_ccc: 0.4424
Epoch 12/50
6295/6295 [==============================] - 50s 8ms/step - loss: 0.5209 - v_loss: 0.1442 - a_loss: 0.1768 - d_loss: 0.2001 - v_ccc: 0.8557 - a_ccc: 0.8233 - d_ccc: 0.8001 - val_loss: 1.5618 - val_v_loss: 0.5546 - val_a_loss: 0.4709 - val_d_loss: 0.5618 - val_v_ccc: 0.4551 - val_a_ccc: 0.5332 - val_d_ccc: 0.4499
Epoch 13/50
6295/6295 [==============================] - 51s 8ms/step - loss: 0.4953 - v_loss: 0.1352 - a_loss: 0.1676 - d_loss: 0.1925 - v_ccc: 0.8648 - a_ccc: 0.8325 - d_ccc: 0.8074 - val_loss: 1.6237 - val_v_loss: 0.5682 - val_a_loss: 0.4815 - val_d_loss: 0.6003 - val_v_ccc: 0.4418 - val_a_ccc: 0.5227 - val_d_ccc: 0.4118
Epoch 14/50
2170/2170 [===========

6295/6295 [==============================] - 50s 8ms/step - loss: 0.8168 - v_loss: 0.2336 - a_loss: 0.2640 - d_loss: 0.3201 - v_ccc: 0.7668 - a_ccc: 0.7362 - d_ccc: 0.6802 - val_loss: 1.5774 - val_v_loss: 0.5831 - val_a_loss: 0.4466 - val_d_loss: 0.5653 - val_v_ccc: 0.4197 - val_a_ccc: 0.5603 - val_d_ccc: 0.4426
Epoch 6/50
6295/6295 [==============================] - 50s 8ms/step - loss: 0.7595 - v_loss: 0.2161 - a_loss: 0.2498 - d_loss: 0.2941 - v_ccc: 0.7840 - a_ccc: 0.7503 - d_ccc: 0.7062 - val_loss: 1.5594 - val_v_loss: 0.5656 - val_a_loss: 0.4487 - val_d_loss: 0.5630 - val_v_ccc: 0.4434 - val_a_ccc: 0.5565 - val_d_ccc: 0.4407
Epoch 7/50
6295/6295 [==============================] - 50s 8ms/step - loss: 0.7236 - v_loss: 0.2026 - a_loss: 0.2378 - d_loss: 0.2835 - v_ccc: 0.7975 - a_ccc: 0.7621 - d_ccc: 0.7168 - val_loss: 1.5984 - val_v_loss: 0.5680 - val_a_loss: 0.4650 - val_d_loss: 0.5772 - val_v_ccc: 0.4376 - val_a_ccc: 0.5370 - val_d_ccc: 0.4270
Epoch 8/50
6295/6295 [==============

6295/6295 [==============================] - 70s 11ms/step - loss: 1.4463 - v_loss: 0.5547 - a_loss: 0.3967 - d_loss: 0.4949 - v_ccc: 0.4452 - a_ccc: 0.6035 - d_ccc: 0.5050 - val_loss: 2.1411 - val_v_loss: 0.6646 - val_a_loss: 0.7910 - val_d_loss: 0.6870 - val_v_ccc: 0.3411 - val_a_ccc: 0.2096 - val_d_ccc: 0.3082
Epoch 2/50
6295/6295 [==============================] - 50s 8ms/step - loss: 1.0711 - v_loss: 0.3522 - a_loss: 0.3153 - d_loss: 0.4039 - v_ccc: 0.6478 - a_ccc: 0.6848 - d_ccc: 0.5963 - val_loss: 1.6160 - val_v_loss: 0.6334 - val_a_loss: 0.4576 - val_d_loss: 0.5334 - val_v_ccc: 0.3709 - val_a_ccc: 0.5473 - val_d_ccc: 0.4658
Epoch 3/50
6295/6295 [==============================] - 50s 8ms/step - loss: 0.9614 - v_loss: 0.2990 - a_loss: 0.2919 - d_loss: 0.3705 - v_ccc: 0.7010 - a_ccc: 0.7081 - d_ccc: 0.6295 - val_loss: 1.5593 - val_v_loss: 0.6048 - val_a_loss: 0.4432 - val_d_loss: 0.5228 - val_v_ccc: 0.4032 - val_a_ccc: 0.5563 - val_d_ccc: 0.4812
Epoch 4/50
6295/6295 [=============

2170/2170 [==============================] - 5s 2ms/step
29
